In [1]:
import sys
from pathlib import Path

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('titanic',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")

Added the following directory to the PYTHONPATH: /home/jdowling/Projects/mlfs-book


In [2]:
import os
import hopsworks
import pandas as pd
from mlfs import config

HopsworksSettings initialized!
2025-12-19 05:53:52,781 INFO: Initializing external client
2025-12-19 05:53:52,782 INFO: Base URL: https://stagingmain.devnet.hops.works:443


ProjectException: Could not find project james

In [ ]:
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")
project = hopsworks.login()
fs = project.get_feature_store()

In [ ]:
titanic_df = pd.read_csv(f"{root_dir}/data/titanic.csv")
titanic_df

### Feature Engineering

 * Impute any missing values for `Age` and `Embarked`

In [ ]:
titanic_df = titanic_df[['PassengerId', 'Sex','Age','Pclass','Fare','Parch','SibSp','Embarked', 'Survived']]
#fill NAs with some imputed values
def_values = {'Age': titanic_df['Age'].mean(), 'Embarked': titanic_df['Embarked'].value_counts().idxmax()}
titanic_df = titanic_df.fillna(value=def_values)
titanic_df

In [ ]:
titanic_fg = fs.get_or_create_feature_group(
    name="titanic",
    version=1,
    primary_key=['PassengerId'],
    description="Titanic passengers dataset"
)

In [ ]:
titanic_fg.insert(titanic_df, wait=True)